In [1]:
import numpy as np
import pickle
from joblib import Parallel, delayed
from scipy import stats
from pybads import BADS
import corner


from psiam_delay_paper_utils import abort_RT_loglike_fn, correct_RT_loglike_fn, wrong_RT_loglike_fn

In [1]:
with open('psiam_data_delay_paper_10K_2.pkl', 'rb') as f:
    psiam_data = pickle.load(f)


choices = psiam_data['choices']
RTs = psiam_data['RTs']
is_act_resp = psiam_data['is_act_resp']
V_A = psiam_data['V_A']
theta_A = psiam_data['theta_A']
V_E = psiam_data['V_E']
theta_E = psiam_data['theta_E']
Z_E = psiam_data['Z_E']
t_stim = psiam_data['t_stim']
t_A_aff = psiam_data['t_A_aff']
t_E_aff = psiam_data['t_E_aff']
t_motor = psiam_data['t_motor']


indices_evid = np.where(is_act_resp == 0)[0]
RTs_evid = RTs[indices_evid].flatten()

indices_act = np.where(is_act_resp == 1)[0]
RTs_act = RTs[indices_act].flatten()

RTs = RTs.flatten()


correct_idx = np.where(choices == 1)[0]
wrong_idx = np.where(choices == -1)[0]
non_abort_idx = np.where(RTs >= t_stim)[0]

correct_non_abort_idx = np.intersect1d(correct_idx, non_abort_idx)
wrong_non_abort_idx = np.intersect1d(wrong_idx, non_abort_idx)

correct_RT = RTs[correct_non_abort_idx]
wrong_RT = RTs[wrong_non_abort_idx]

abort_idx = np.where(RTs < t_stim)[0]
abort_RT = RTs[abort_idx]


print(f"V_A: {V_A}")
print(f"theta_A: {theta_A}")
print(f"V_E: {V_E}")
print(f"theta_E: {theta_E}")
print(f"t stim is {t_stim}")
print(f"t_A_aff: {t_A_aff}, t_E_aff: {t_E_aff}, t_motor: {t_motor}")

print(f"Num of AI process: {is_act_resp.sum()}/{len(is_act_resp)}")
print(f'Number of aborts = {(RTs < t_stim).sum()}')



NameError: name 'pickle' is not defined

In [3]:
def psiam_neg_loglike_fn(params):
    V_A, theta_A, V_E, theta_E, Z_E, t_A_aff, t_E_aff, t_motor = params

    K_max = int(10)
    n_jobs = -1

    abort_loglike = sum(Parallel(n_jobs=n_jobs)(delayed(abort_RT_loglike_fn)(t, V_A, theta_A, V_E, theta_E, Z_E, K_max, t_A_aff, t_E_aff, t_stim, t_motor) for t in abort_RT))
    correct_loglike = sum(Parallel(n_jobs=n_jobs)(delayed(correct_RT_loglike_fn)(t, V_A, theta_A, V_E, theta_E, Z_E, K_max, t_A_aff, t_E_aff, t_stim, t_motor) for t in correct_RT))
    wrong_loglike = sum(Parallel(n_jobs=n_jobs)(delayed(wrong_RT_loglike_fn)(t, V_A, theta_A, V_E, theta_E, Z_E, K_max, t_A_aff, t_E_aff, t_stim, t_motor) for t in wrong_RT))


    loglike = abort_loglike + correct_loglike + wrong_loglike

    if np.isnan(loglike):
        raise ValueError('loglike is nan')
    if np.isinf(loglike):
        raise ValueError('loglike is inf')
    
    return -loglike
    

In [4]:
V_A_bounds = [0.1, 2]; V_A_plausible_bounds = [0.5, 1.5]
theta_A_bounds = [1, 3]; theta_A_plausible_bounds = [1.5, 2.5]
V_E_bounds = [-2, 2]; V_E_plausible_bounds = [-1.5, 1.5]
theta_E_bounds = [0.1, 3]; theta_E_plausible_bounds = [0.5,2.5]
Z_E_bounds = [-0.5, 0.5]; Z_E_plausible_bounds = [-0.2, 0.2]
t_A_aff_bounds = [0.01, 0.06]; t_A_aff_plausible_bounds = [0.015, 0.05]
t_E_aff_bounds = [0.01, 0.06]; t_E_aff_plausible_bounds = [0.02, 0.05]
t_motor_bounds = [0.01, 0.06]; t_motor_plausible_bounds = [0.03, 0.06]


In [5]:
lb = np.array([V_A_bounds[0], theta_A_bounds[0], V_E_bounds[0], theta_E_bounds[0], Z_E_bounds[0], 
               t_A_aff_bounds[0], t_E_aff_bounds[0], t_motor_bounds[0]])
ub = np.array([V_A_bounds[1], theta_A_bounds[1], V_E_bounds[1], theta_E_bounds[1], Z_E_bounds[1], 
               t_A_aff_bounds[1], t_E_aff_bounds[1], t_motor_bounds[1]])
plb = np.array([V_A_plausible_bounds[0], theta_A_plausible_bounds[0], V_E_plausible_bounds[0], theta_E_plausible_bounds[0], Z_E_plausible_bounds[0], 
                t_A_aff_plausible_bounds[0], t_E_aff_plausible_bounds[0], t_motor_plausible_bounds[0]])
pub = np.array([V_A_plausible_bounds[1], theta_A_plausible_bounds[1], V_E_plausible_bounds[1], theta_E_plausible_bounds[1], Z_E_plausible_bounds[1], 
                t_A_aff_plausible_bounds[1], t_E_aff_plausible_bounds[1], t_motor_plausible_bounds[1]])

np.random.seed(42)  
V_A_0 = np.random.uniform(plb[0], pub[0])
theta_A_0 =  np.random.uniform(plb[1], pub[1])
V_E_0 = np.random.uniform(plb[2], pub[2])
theta_E_0 = np.random.uniform(plb[3], pub[3])
Z_0 = np.random.uniform(plb[4], pub[4])
t_A_aff_0 = np.random.uniform(plb[5], pub[5])
t_E_aff_0 = np.random.uniform(plb[6], pub[6])
t_motor_0 = np.random.uniform(plb[7], pub[7])

x0 = np.array([V_A_0, theta_A_0, V_E_0, theta_E_0, Z_0, t_A_aff_0, t_E_aff_0, t_motor_0]);


In [6]:
options = {'display': 'off'}
bads = BADS(psiam_neg_loglike_fn, x0, lb, ub, plb, pub, options=options)
optimize_result = bads.optimize()
x_min = optimize_result['x']


print(f'Est. V_A = {x_min[0]}, True V_A = {V_A}')
print(f'Est. theta_A = {x_min[1]}, True theta_A = {theta_A}')
print(f'Est. V_E = {x_min[2]}, True V_E = {V_E}')
print(f'Est. theta_E = {x_min[3]}, True theta_E = {theta_E}')
print(f'Est. Z = {x_min[4]}, True Z = {Z_E}')
print(f'Est. t_A_aff = {x_min[5]}, True t_A_aff = {t_A_aff}')
print(f'Est. t_E_aff = {x_min[6]}, True t_E_aff = {t_E_aff}')
print(f'Est. t_motor = {x_min[7]}, True t_motor = {t_motor}')

bads:TooCloseBounds: For each variable, hard and plausible bounds should not be too close. Moving plausible bounds.
Est. V_A = 1.002268140957753, True V_A = 1
Est. theta_A = 2.018140455422092, True theta_A = 2
Est. V_E = 1.224765021220989, True V_E = 1.2
Est. theta_E = 2.014954265141057, True theta_E = 2
Est. Z = -0.018122722803207127, True Z = 0
Est. t_A_aff = 0.04452589218115613, True t_A_aff = 0.02
Est. t_E_aff = 0.020749976264883688, True t_E_aff = 0.03
Est. t_motor = 0.027316925668284035, True t_motor = 0.05
